<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/notebooks/inference/tpu_index_search_million_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!gdown --id "1-23aNm7j0bnycvyd_OaQfofVYPTewgOI"   # abstract vectors
!gdown --id "1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX"   # TitlesAbstractsEmbedIds
!wget 'https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar'
!tar -xvf 'scibert_scivocab_uncased.tar'

Downloading...
From: https://drive.google.com/uc?id=1-23aNm7j0bnycvyd_OaQfofVYPTewgOI
To: /content/AbstractSimVectors.npy
2.59GB [00:21, 121MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX
To: /content/TitlesAbstractsEmbedIds.json.gzip
397MB [00:03, 101MB/s] 
--2019-12-29 16:17:02--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.218.104
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.218.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 421.96M  52.0MB/s    in 8.5s    

2019-12-29 16:17:11 (49.7 MB/s) - ‘scibert_scivocab_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scib

In [0]:
%tensorflow_version 2.x

!pip install transformers --quiet
!pip install tpu-index

TensorFlow 2.x selected.
     |████████████████████████████████| 450kB 3.4MB/s 
     |████████████████████████████████| 860kB 17.1MB/s 
     |████████████████████████████████| 1.0MB 22.7MB/s 


In [0]:
import numpy as np
import tensorflow as tf
from time import time
from transformers import BertTokenizer
import pandas as pd
from pprint import pprint
from tpu_index import TPUIndex

print('TensorFlow:', tf.__version__)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: 10.92.98.170:8470


INFO:tensorflow:Initializing the TPU system: 10.92.98.170:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TensorFlow: 2.1.0-rc1


In [0]:
abstract_embeddings = np.load('AbstractSimVectors.npy')
abstract_embeddings.shape

(1262996, 512)

## Initialize and Build the Index

In [0]:
index = TPUIndex(num_tpu_cores=8)
index.create_index(abstract_embeddings)

Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7


### The first search runs relatively slow, but the searches that follow run blazing fast!

In [0]:
dummy_vector = tf.random.normal([1, 512])
s = time()
_ = index.search(dummy_vector, distance_metric='cosine', top_k=20)
e = time()
print('Warmup search time :', np.round(e-s, 3), 'secs')

Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7
Warmup search time : 41.381 secs


  ### Running the search
  - We use BERT model to encode paper abstracts as 512 dim vectors
  - We then use this embedding vector to search against our index, which has 1.26 million computer science papers indexed.
  - Each search takes around 180ms, after 1 or 2 warmup searches.

In [0]:
model = tf.saved_model.load('gs://tfworld/saved_models')

In [0]:
tokenizer = BertTokenizer(vocab_file='scibert_scivocab_uncased/vocab.txt')

df = pd.read_json('TitlesAbstractsEmbedIds.json.gzip', compression = 'gzip')
embed2Title = pd.Series(df['title'].values,index=df['EmbeddingID']).to_dict()
embed2Abstract = pd.Series(df['paperAbstract'].values,index=df['EmbeddingID']).to_dict()

 #### Select a random paper id from the corpus, or paste any random paper abstract

In [0]:
embed_id = 108712
title = embed2Title[embed_id]         # Not used in search
abstract = embed2Abstract[embed_id]   # Put your abstract(CS related) here or use our corpus

abstract_encoded = tokenizer.encode(abstract, max_length=512, pad_to_max_length=True)
abstract_encoded = tf.constant(abstract_encoded, dtype=tf.int32)[None, :]
print('Title : ')
pprint(title)
print('\nAbstract : ')
pprint(abstract)

Title : 
('Open circuit fault diagnosis for the power electronic converter stages using '
 'ANFIS algorithm')

Abstract : 
('This paper demonstrates a fault detection and location for the open circuit '
 '(O.C) faults in the all power electronic converter stages including; the '
 'three-phase rectifier, the dc-dc converter, and finally the single-phase '
 'inverter. The proposed fault diagnosis is based on the adaptive neuro-fuzzy '
 'inference system (ANFIS) algorithm. The inputs to the ANFIS unit are the '
 'voltage and the current measurements of the power converter stages after '
 'they undergo several signal processing operations so that they could '
 'efficiently reflect the fault occurrence and behavior. The output of the '
 'ANFIS unit is utilized as an index in order to identify the O.C fault in the '
 'power converter. Then, it locates the fault within the three different '
 'stages of the converter.')


In [0]:
s = time()
bert_output = model(abstract_encoded)
xq = tf.nn.l2_normalize(bert_output, axis=1)
e_p = time()
prediction_time = e_p - s

In [0]:
s_s = time()
aD, aI = index.search(xq, distance_metric='cosine', top_k=5)
e_s = time()
search_time = e_s - s_s
print('\n'*2)
print('Prediction time  :', np.round(prediction_time, 3), 'secs')
print('Search time      :', np.round(search_time, 3), 'secs')
print('Total time       :', np.round(prediction_time + search_time, 3), 'secs')

print('\n'*2)
print('*'*80)
for i in range(len(aI)):
  print('Abstract : ')
  pprint(embed2Abstract[aI[i]])
  print('*'*80)
print('\nNeighbours       :', aI )
print('Distances        :', np.round(aD, 4))

Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7



Prediction time  : 0.003 secs
Search time      : 0.163 secs
Total time       : 0.166 secs



********************************************************************************
Abstract : 
('This paper demonstrates a fault detection and location for the open circuit '
 '(O.C) faults in the all power electronic converter stages including; the '
 'three-phase rectifier, the dc-dc converter, and finally the single-phase '
 'inverter. The proposed fault diagnosis is based on the adaptive neuro-fuzzy '
 'inference syste